# Maths Lab - 6

## Teammates:
#### Akshita Potdar - 702768460
#### Kalyani Khandait - 702768391
#### Logesh Gangadharan - 702723890


##  Problem -1

Use PyTorch to find a local minimum of the function
f (x, y) = 1 − x − y + x2 + 2y2.

In [1]:
import torch

# Defining  the function
def f(x):
    return 1 - x[0] - x[1] + x[0]**2 + 2*x[1]**2

# Initialize x and y with requires_grad=True to track gradients
x = torch.tensor([1.0, 1.0], requires_grad=True)  # Start at (1, 1)

# Use the Adam optimizer  (lr = learning rate)
optimizer = torch.optim.Adam([x], lr=0.01)


maxit = 2000 # Maximum number of optimization steps
i = 0 # Iteration counter
grad = 1e6 # Keeps track of derivative with respect to x


# Optimization loop
while i < maxit and abs(grad) > 1e-3:
    i+=1
    optimizer.zero_grad()  # Clear the gradients

    y = f(x)  # Calculate the function value
    y.backward()  # Compute the  gradients
    optimizer.step()     # update the parameters (here: value of x)
    grad_norm = x.grad.norm().item()

    # Print the current state
    # print(f"Iter: {i+1}, x = [{x[0].item():.6f}, {x[1].item():.6f}], f(x) = {y.item():.6f}, grad_norm = {grad_norm:.6f}")

# Final output
print(f"Final x = [{x[0].item():.6f}, {x[1].item():.6f}]")



Final x = [0.500000, 0.250000]


##  Problem -2

In this problem, you will train a neural network to predict the label
of the flowers in the Iris dataset that we worked with before

In [2]:
import torch
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target.reshape(-1, 1)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert labels using one-hot encoding
encoder = OneHotEncoder(sparse_output=False)  # Ensures that the output is dense
y_encoded = encoder.fit_transform(y)  # Output is dense by default, no need to convert to array

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

# Convert data to torch tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test)


In [3]:
import torch.nn as nn

class SimpleNN(nn.Module):
    def __init__(self, input_features, hidden_units, output_features):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_features, hidden_units)  # Input to hidden layer
        self.relu = nn.ReLU()                                          # ReLU activation
        self.fc2 = nn.Linear(hidden_units, output_features)  # Hidden to output layer

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Initializing  the model
model = SimpleNN(input_features=4, hidden_units=10, output_features=3)


In [4]:
# Setting loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [100/1000], Loss: 0.0827
Epoch [200/1000], Loss: 0.0503
Epoch [300/1000], Loss: 0.0453
Epoch [400/1000], Loss: 0.0436
Epoch [500/1000], Loss: 0.0429
Epoch [600/1000], Loss: 0.0425
Epoch [700/1000], Loss: 0.0422
Epoch [800/1000], Loss: 0.0421
Epoch [900/1000], Loss: 0.0419
Epoch [1000/1000], Loss: 0.0418


In [5]:
# Testing the model
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    _, predicted = torch.max(test_outputs, 1)
    _, labels = torch.max(y_test_tensor, 1)
    accuracy = (predicted == labels).float().mean()*100
    print(f'Accuracy: {accuracy:.4f}')


Accuracy: 100.0000
